In [1]:
from OddoDownload import OddoDownload
import pandas as pd
import numpy as np

In [5]:
conn_params = {
    'ODOO_USERNAME' : 'acceso@asalvo.cl',
    'ODOO_PASSWORD' : '1234',
    'ODOO_HOSTNAME' : 'sinergia-lab-cencorep-ambiente-test-9601418.dev.odoo.com',
    'ODOO_DATABASE' : 'sinergia-lab-cencorep-ambiente-test-9601418'
}
odoo = OddoDownload(conn_params)

modelo = 'x_materialidad'
filtros = ['|',('x_studio_mat','=','CARTÓN'),('x_name','=','CAJA')]
campos = ['x_studio_mat','x_name']
header = ['campo1','campo2']

# for i in tqdm(range(10)):
    # odoo.getDataFromModel(modelo,filtros,campos)

odoo.getDataFromModel(modelo,filtros,campos)
odoo.downloadExcel('excel test')


Descargando data
.
.
.
done
Generando archivo


In [6]:
len(odoo.resultadoBusqueda)

138531

In [4]:
odoo.resultadoBusqueda

,id,x_studio_mat,x_name
0,61149,CARTÓN,CAJA
1,61156,CARTÓN,CAJA
2,61163,CARTÓN,CAJA
3,61170,CARTÓN,CAJA
4,61177,CARTÓN,CAJA
...,...,...,...
5291,1441215,CARTÓN,CAJA
5292,1441219,CARTÓN,CAJA
5293,1441223,CARTÓN,CAJA
5294,1441227,CARTÓN,CAJA
